# Script to plot EKE and wind stress over a chosen box

This script is designed so that we can select an individual box and compute:
 * The EKE for each member over that box
 * The ensemble mean EKE over that box
 * The wind stress over that box
 * The lagged cross-correlation between EKE and stress
 
It is slow, even distributed over multiple cores, so be patient!

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.path as mpath
import xarray as xr
import numpy as np
import IPython.display
import pandas as pd
import cmocean as cm
import cartopy.crs as ccrs
import cartopy.feature as cft
import os 

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client

In [2]:
client = Client(n_workers=4)
client

Client Scheduler: tcp://127.0.0.1:34568 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [3]:
land_50m = cft.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='black',
                                        facecolor='gray',
                                        linewidth=1.0)

In [4]:
HardDisk = '/g/data/v45/amh157/OCCIPUT/'
EnsembleDir = 'geouv/'

In [5]:
## To load the EKE climatology
climfilename = HardDisk+EnsembleDir+'eke_climatology.nc'
eke_month = xr.open_dataset(climfilename).EKE

In [6]:
# function to process and plot eke and tau, and NOT save as files
def proc_plot_eke_box(desc, lon1,  lon2, lat1,  lat2, ax_p, ax_b, ax_pt):
    nfiles = 5 #0
    nn=0
    start = pd.datetime(1980,1,1) ## start time for plotting
    for ii in range(1,nfiles+1):
        nn+=1
        i00 = '%03d' % ii
        Member = 'e'+i00
        ekefilename = HardDisk+EnsembleDir+Member+'/eke_'+i00+'_????.nc'
        #outfilename = HardDisk+EnsembleDir+Member+'/eke_'+desc+'_'+i00+'.nc'
        
        eke = xr.open_mfdataset(ekefilename).EKE
        #print(ekefilename+' -> '+outfilename)
    
        eke = eke.sel(lon=slice(lon1,lon2)).sel(lat=slice(lat1,lat2)).mean('lon').mean('lat')
        eke = eke.rolling(time=13,center=True).mean()
        eke.plot(ax=ax_p,color=[0.5, 0.5, 0.5],linewidth=0.5)
        #eke.to_netcdf(outfilename)
        
        if nn==1:
            eke_sum = eke
        else:
            eke_sum = eke_sum + eke
            
        print(ekefilename)
            
    eke_sum = eke_sum/nn
    eke_sum.plot(ax=ax_p,color='r',linewidth=1.5)
    ax_p.set_xlabel('Time')
    ax_p.set_ylabel('EKE',color=col)
    
    taufilename = HardDisk+EnsembleDir+'tau/'+'tau_????.nc'
    tau = xr.open_mfdataset(taufilename).tau
    
    #outfilename = HardDisk+EnsembleDir+'tau/'+'tau_'+desc+'.nc'
    #print(taufilename+' -> '+outfilename)
    tau = tau.sel(lon=slice(lon1,lon2)).sel(lat=slice(lat1,lat2)).mean('lon').mean('lat')
    tau = tau.rolling(time=13,center=True).mean()
    tau.plot(ax=ax_pt,color='k',linewidth=1,linestyle='--')
    ax_pt.set_ylim(ymin=0.08)
    ax_pt.set_ylabel('Wind Stress')
    #tau.to_netcdf(outfilename)

In [ ]:
%%time
plt.figure(figsize=(14, 6))
ax = plt.axes([0.05,0.1,0.4,0.8],projection=ccrs.SouthPolarStereo())
ax.set_extent([-287, 73, -80, -30], crs=ccrs.PlateCarree())

# Compute a circle in axes coordinates, which we can use as a boundary
# for the map. We can pan/zoom as much as we like - the boundary will be
# permanently circular.
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)

ax.add_feature(land_50m)
ax.gridlines()
p1 = eke_month.mean('month').plot(cmap=cm.cm.speed,vmax=0.16,add_colorbar=False,transform=ccrs.PlateCarree())
plt.title('EKE Climatology')

ax_c = plt.axes([0.02,0.7,0.01,0.2])
plt.colorbar(p1,cax=ax_c,orientation='vertical',extend='max')

ax2 = plt.axes([0.5,0.2,0.4,0.6])
ax3 = ax2.twinx()
proc_plot_eke_box('Agulhas Retroflection', 12,  35, -45,  -33, ax2, ax, ax3)

plt.savefig('temp.png')

/g/data/v45/amh157/OCCIPUT/geouv/e001/eke_001_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e002/eke_002_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e003/eke_003_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e004/eke_004_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e005/eke_005_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e006/eke_006_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e007/eke_007_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e008/eke_008_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e009/eke_009_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e010/eke_010_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e011/eke_011_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e012/eke_012_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e013/eke_013_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e014/eke_014_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e015/eke_015_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e016/eke_016_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e017/eke_017_????.nc
/g/data/v45/amh157/OCCIPUT/geouv/e018/eke_018_????.nc
/g/data/v45/amh157/OCCIPUT/g